# Analysis Notebook for the scraper project

In [37]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import os
import sys
import re

In [39]:
pd.options.display.max_columns = None

In [44]:
root = Path(os.getcwd()).parent
fname = root / 'data' / 'scraper_df.csv'
df=pd.read_csv(fname)
df

,Unnamed: 0,data-advert-position,class,href,data-attr-href,alt,data-trace,data-category,data-label,data-actions,owner,timestamp,text,details,category1,bid_ask,rent_buy,rooms,cost,address,duration
0,Wunderschönes Co-Living Haus mit 38 Suiten in ...,2042131_1220_cff62a975baadb14ee9b9fdec831d6f1,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,Wunderschönes Co-Living Haus mit 38 Suiten in ...,NaN,ContentList,ContentTitle,click,johannespeter,2020-02-09 10:53:00,"Ich glaube, dass zusammen einfach schöner ist ...",Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,1'095.00,"Marktgasse 19, 8038 Kloten",unbefristet
1,WG-Zimmer in Uster,2072842_1220_6b9bf9cf83d6f7104cda89917b4ca96c,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,WG-Zimmer in Uster,NaN,ContentList,ContentTitle,click,nicolerohr,2020-10-17 20:17:00,"In 3.5 Zimmer Attika Terrassenwohnung, 1 zimme...",Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,1'200.00,"Talweg 145, 8610 Uster",unbefristet
2,Süsse Zimmer in einem Grossen Einfamilienhaus,2072840_1220_af70092015d5a4d198d13c1b7a645025,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,Süsse Zimmer in einem Grossen Einfamilienhaus,NaN,ContentList,ContentTitle,click,Leon Zehr,2020-10-17 19:15:00,"Hallo zusammen,Wir sind auf der Suche nach ein...",Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,800.00,"Schauenbergstr. 17, 8046 Zürich",befristet
3,"suche gepflegtes wg-wohnen, biete haushalthilf...",2071434_1220_0bfaae68b0197724045843cffb9b73df,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,"suche gepflegtes wg-wohnen, biete haushalthilf...",NaN,ContentList,ContentTitle,click,air,2020-10-17 00:00:00,"kultivierter, solventer er, wg-erfahrensucht g...",Biete / Suche / Tausche: Wohnung / Zimmer: Suc...,Wohnung / Zimmer,Suche,Wohnung / Zimmer,1.0,unknown,8000 zürich,unbefristet
4,15qm Zimmer in heller & grosser 4er-WG,2072828_1220_463403d49ffd059d8b4c9437666c4297,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,15qm Zimmer in heller & grosser 4er-WG,NaN,ContentList,ContentTitle,click,Clement O.,2020-10-17 17:08:00,"Das Zimmer: ist 15qm gross, hell und ruhig gel...",Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,790.00,"Neunbrunnenstrasse , 8050 Zürich",unbefristet
5,Zimmer in Zürich Altstetten zu vermieten,2072816_1220_4ebf70ce96a6d5a47be1df1f9d3aea08,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,Zimmer in Zürich Altstetten zu vermieten,NaN,ContentList,ContentTitle,click,Gorilla333,2020-10-17 15:52:00,Zimmer zu vermieten13m2 / Fr. 800.-Gemeinschaf...,Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,800.00,"Bachmattweg, 8048",unbefristet
6,"Cozy Studio near Schmiede Wiedikon, 1405CHF/month",2072688_1220_00893ff66458a536d3f4d0983d8f3073,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,"Cozy Studio near Schmiede Wiedikon, 1405CHF/month",NaN,ContentList,ContentTitle,click,Songmin Zhao,2020-10-17 14:37:00,"Hi,I am moving out of my studio and would like...",www.newhome.ch Biete / Suche / Tausche: Wohnun...,Wohnung / Zimmer,Biete,Wohnung / Zimmer,1.0,1'405.00,8003 Zürich,unknown
7,WG-ZIMMER IN HAUS IM GRÜNEN,1757116_1220_70d5b5e70a66698203de262b39be70a8,[],https://www.ronorp.net/zuerich/immobilien/wg-z...,https://www.ronorp.net/zuerich/immobilien/wg-z...,WG-ZIMMER IN HAUS IM GRÜNEN,NaN,ContentList,ContentTitle,click,dees,2020-10-17 00:00:00,WIR SUCHEN EINE WUNDERVOLLE SEELE FÜR EINES OD...,Biete / Suche / Tausche: Wohnung / Zimmer: Bie...,Wo

In [45]:
len(df)

16

In [15]:
df.loc[0,'details']

"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Agglo Vertragsart: unbefristet Zimmer: 1Kosten: 1'095.00Adresse: Marktgasse 19, 8038 Kloten Kontakt: i***@t*************hDie vollen Kontaktinformationen sind nur für Ron Orp Leser sichtbar. Hier gleich kostenlos anmelden. \t \t\t \t\tAchtung: Überweise nie eine Kaution an Personen, deren Wohnung du nie gesehen hast oder weil sie gerade nicht in der Schweiz weilen.\t \t mehr SCHREIB MIR! Anzeige Crowdfunding Workshop Food ZH Aufstrebende Gastronomen aufgepasst! Am Crowdify Workshop erhalten ambitionierte Feinschmecker den Masterplan für eine erfolgreiche Crowdfunding-Aktion. Mehr Infos Werbeanzeige WG Folgen ♥ ♥ ♥ Unterstütze Ron ♥ ♥ ♥Ich bin dein Sprachrohr für deine Stadt und dabei stets unabhängig. Daher freue ich mich über alle, die bereit sind, einen kleinen, finanziellen Beitrag an mein Tun zu leisten. Kommentare1 auf Merkliste"

In [14]:
df.loc[1,'details']

"www.newhome.ch Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Zimmer: 1.0Kosten: 1'405.00Adresse: 8003 Zürich Kontakt:Die vollen Kontaktinformationen sind nur für Ron Orp Leser sichtbar. Hier gleich kostenlos anmelden. Original Inserat auf: \t \t\t \t\tAchtung: Überweise nie eine Kaution an Personen, deren Wohnung du nie gesehen hast oder weil sie gerade nicht in der Schweiz weilen.\t \t mehr SCHREIB MIR! Anzeige Crowdfunding Workshop Food ZH Aufstrebende Gastronomen aufgepasst! Am Crowdify Workshop erhalten ambitionierte Feinschmecker den Masterplan für eine erfolgreiche Crowdfunding-Aktion. Mehr Infos Werbeanzeige WG Folgen ♥ ♥ ♥ Unterstütze Ron ♥ ♥ ♥Ich bin dein Sprachrohr für deine Stadt und dabei stets unabhängig. Daher freue ich mich über alle, die bereit sind, einen kleinen, finanziellen Beitrag an mein Tun zu leisten. 0 auf Merkliste"

['C:\\coding\\wgscraper\\notebooks',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\DLLs',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper',
 '',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Anaconda3\\envs\\wgscraper\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\cbole\\.ipython',
 WindowsPath('C:/coding/wgscraper/src'),
 WindowsPath('C:/coding/wgscraper/src'),
 'C:\\coding\\wgscraper\\src']

In [41]:

    
regex = {r'category1': r'Biete \/ Suche \/ Tausche: (.+?):',
         r'bid_ask': r'Biete \/ Suche \/ Tausche: .+?: (.+?) ',
         r'rent_buy': r'Mieten \& Kaufen: (.+?) ',
         r'rooms': r'Zimmer: ([^a-zA-Z ]+)',
         r'cost': r'Kosten: ([^a-zA-Z ]+)',
         r'address': r'Adresse: (.*?)Kontakt',
         r'duration': r'Vertragsart: (.*?) ',
         }
for i in [0,1]:
    d = {}
    for k, v in regex.items():
        prop = re.search(v, df.loc[i,'details'])
        if prop is not None:
            prop = prop.groups(0)[0]
        else:
            prop = 'unknown'
        print(prop)

Wohnung / Zimmer
Biete
Mieten
1
1'095.00
Marktgasse 19, 8038 Kloten 
unbefristet
Wohnung / Zimmer
Biete
Mieten
1.0
1'405.00
8003 Zürich 
unknown
